<a href="https://colab.research.google.com/github/Git-Codder/LRFD_Model/blob/main/Data_Preprocessing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Enabling TPU in Google Colab

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

#Mout Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Install Dependencies 

In [ ]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.8 MB/s 


In [ ]:
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt
import numpy as np

#unzip the file

In [ ]:
!unzip /content/drive/MyDrive/LRFD/Data_Images.zip -d /content/drive/MyDrive/LRFD/

#List All files in directory

In [ ]:
import os
from PIL import Image

info = []
for root, __, files in os.walk("/content/drive/MyDrive/LRFD/images"):
  for f in files:
      if f.endswith(".jpg"):
         info.append({
                      "path":os.path.join(root, f),
                      "img": Image.open(os.path.join(root, f))
                     # "foldername": os.path.dirname(root)
                     })
         
len(info)


5000

In [ ]:
# info[0]['path'] 

#Deep Copy

In [ ]:
from copy import deepcopy
labeled_info = deepcopy(info)

#MTCNN Detector

In [ ]:
detector = MTCNN()
i = 0
for i in range(len(labeled_info)):
  img1 = cv2.cvtColor(cv2.imread(labeled_info[i]['path']), cv2.COLOR_BGR2RGB)
  temp = detector.detect_faces(img1)
  if(len(temp) == 1):
    cord = temp[0]['box']
    labeled_info[i]['label'] = 'face'
    labeled_info[i]['points'] = [[cord[0], cord[1]], [cord[0]+cord[2], cord[1]+cord[3]]]
  elif(len(temp) == 0):
    labeled_info[i]['label'] = 'noface'
    labeled_info[i]['points'] = [[0,0],[0,0]]
  if(i%500 == 0):
    print(i)

len(labeled_info)
    

0
500
1000
1500
2000
2500
3000
3500
4000
4500


5000

In [ ]:
labeled_info_1 = deepcopy(labeled_info)

In [ ]:
# labeled_info_1[0]

#predict Test Image

In [ ]:
img = cv2.cvtColor(cv2.imread(info[0]['path']), cv2.COLOR_BGR2RGB)
detector = MTCNN()
cord = detector.detect_faces(img)[0]['box'];

In [ ]:
line_red = cv2.rectangle(img,(cord[0],cord[1]),(cord[0]+cord[2],cord[1]+cord[3]),(255,0,0),1)
plt.imshow(line_red)

#Remove 'img' attribute from object

In [ ]:
for items in labeled_info_1:
        del items['img']
labeled_info_1[0]

{'label': 'face',
 'path': '/content/drive/MyDrive/LRFD/images/img_884_4962.jpg',
 'points': [[16, 6], [36, 29]]}

#Read FIle and Preprocessing

In [ ]:
import json
def create_json(stList):
    with open("/content/drive/MyDrive/LRFD/labeled_info_1.json", "w") as file:
        json.dump([ob for ob in stList], file)

In [ ]:
create_json(labeled_info_1)

In [ ]:
import pandas as pd
labeled_info = pd.read_json('/content/drive/MyDrive/LRFD/labeled_info_1.json')
labeled_info

,path,label,points
0,/content/drive/MyDrive/LRFD/images/img_884_496...,face,"[[16, 6], [36, 29]]"
1,/content/drive/MyDrive/LRFD/images/img_245_460...,noface,"[[0, 0], [0, 0]]"
2,/content/drive/MyDrive/LRFD/images/img_1072_14...,noface,"[[0, 0], [0, 0]]"
3,/content/drive/MyDrive/LRFD/images/img_620_391...,face,"[[9, 5], [21, 25]]"
4,/content/drive/MyDrive/LRFD/images/img_643_207...,face,"[[23, 7], [34, 21]]"
...,...,...,...
4995,/content/drive/MyDrive/LRFD/images/img_908_434...,face,"[[26, 11], [39, 24]]"
4996,/content/drive/MyDrive/LRFD/images/img_482_487...,face,"[[20, 10], [32, 26]]"
4997,/content/drive/MyDrive/LRFD/images/img_1068_11...,face,"[[23, 6], [40, 24]]"
4998,/content/drive/MyDrive/LRFD/images/img_589_263...,face,"[[29, 8], [39, 21]]"


In [ ]:
type(labeled_info)

pandas.core.frame.DataFrame

#Create Json file for each image & Move images in right folder

In [ ]:
# folder = ['30','70','110','200']
# for fold in folder:
#   cnt = 1;
#   os.mkdir("/content/drive/MyDrive/LRFD/Image_data/" + fold + "/noface")
#   os.mkdir("/content/drive/MyDrive/LRFD/Image_data/" + fold + "/labels")
#   os.mkdir("/content/drive/MyDrive/LRFD/Image_data/" + fold + "/img")
#   for root, __, files in os.walk("/content/drive/MyDrive/LRFD/Image_data/" + fold):
#     for f in files:
#         if f.endswith(".jpg"):
#           path=os.path.join(root, f)
#           split_string = path.split(".",1)
#           split_string_json = path.split("/")
          
#           newnameJson = ""
#           for v in range(len(split_string_json)-3):
#             newnameJson += split_string_json[v] + "/"
#           newnameJson += "labels/"

#           newnameJson += split_string_json[len(split_string_json)-1].split(".",1)[0] + "_" + str(cnt) + ".json"
#           newnameImg = split_string[0] +"_"+ str(cnt) + ".jpg"

#           result = labeled_info.loc[(labeled_info['path'] == path)]
#           if(result['label'].iloc[0] == 'noface'):
#               !mv $path $newnameImg
#               dest = "/content/drive/MyDrive/LRFD/Image_data/" + fold + "/noface"
#               !mv $newnameImg $dest
#               print("removed file wiht path : " + path)
#           else:
#             !mv $path $newnameImg
#             dest = "/content/drive/MyDrive/LRFD/Image_data/" + fold + "/img"
#             !mv $newnameImg $dest
#             obj = []
#             obj.append({
#                 "label":"face",
#                 "points":result['points'].iloc[0]
#             })
            
#             with open(newnameJson, "w") as file:
#               json.dump([ob for ob in obj], file)
#           cnt += 1



In [ ]:

cnt = 1;
os.mkdir("/content/drive/MyDrive/LRFD/Image_data/noface")
os.mkdir("/content/drive/MyDrive/LRFD/Image_data/labels")
os.mkdir("/content/drive/MyDrive/LRFD/Image_data/img")
for root, __, files in os.walk("/content/drive/MyDrive/LRFD/images/"):
  for f in files:
      if f.endswith(".jpg"):
        path=os.path.join(root, f)
        split_string = path.split(".",1)
        split_string_json = path.split("/")
        
        newnameJson = ""
        for v in range(len(split_string_json)-2):
          newnameJson += split_string_json[v] + "/"
        newnameJson += "/Image_data/labels/"

        newnameJson += split_string_json[len(split_string_json)-1].split(".",1)[0] + "_" + str(cnt) + ".json"
        # print(newnameJson)
        newnameImg = split_string[0] +"_"+ str(cnt) + ".jpg"

        result = labeled_info.loc[(labeled_info['path'] == path)]
        if(result['label'].iloc[0] == 'noface'):
            !mv $path $newnameImg
            dest = "/content/drive/MyDrive/LRFD/Image_data/noface"
            !mv $newnameImg $dest
            print("removed file wiht path : " + path)
        else:
          !mv $path $newnameImg
          dest = "/content/drive/MyDrive/LRFD/Image_data/img"
          !mv $newnameImg $dest
          obj = []
          obj.append({
              "label":"face",
              "points":result['points'].iloc[0]
          })
          
          with open(newnameJson, "w") as file:
            json.dump([ob for ob in obj], file)
        cnt += 1



In [ ]:
!zip -rv labeld_images_new "/content/drive/MyDrive/LRFD/Image_data"

In [ ]:
!mv "/content/labeld_images_new.zip" "/content/drive/MyDrive/LRFD/"